In [19]:
import json
from pprint import pprint
from PIL import ImageColor
from xml.etree import ElementTree as ET

# 讀取投票結果
with open("voting_results.json","r",encoding="utf=8") as file:
    voting_results = json.load(file)

# 以縣市為迭代單位，計算各黨得票數    
parties = {"dpp","kmt", "tpp"}
votes ={}

for city in voting_results["district_total"].keys(): #取出「"district_total"」的城市名稱給接下來用
    party_votes = {party : voting_results[party][city] for party in parties} #party 是所有鍵ㄉ名稱
    votes[city] = party_votes #每個城市寫一個新字典
    
# pprint(votes) # 檢查用

# 再次以縣市為迭代單位，用 tuple (city, total_votes) ，從初始的字典形式取得「城市名稱」與「該城市得票總數」
# 用迴圈跑三個政黨在縣市裡的得票數÷該縣市總數，做成新的字典
# 把新字典更新到 percentages = {} 裡面

percentages = {}

for (city, total_votes) in voting_results["district_total"].items(): # 用總票數字典的鍵當地名、值當總票數分母
    party_percentages = {party : voting_results[party][city]/total_votes*100 for party in parties}
    percentages.update({city: party_percentages})

# pprint(percentages) # 檢查用


# 計算得票比例差距，並找出差異最大的城市
all_difference={}

for (city, party_percentage) in percentages.items(): # 以 tuple 形式取用％數字典的資料，前者是城市，後者是各黨得票％數字典
    percentage_list = list(party_percentage.items()) # 每一個城市的三筆黨派％數做成串列
    sorted_parties = sorted(percentage_list, key = lambda x: x[1], reverse = True) # 串列排序魔法
    winner = sorted_parties[0][0] # 取出第一名[0]，再取出第一名的名字[0]
    votes_difference = round(sorted_parties[0][1]- sorted_parties[1][1],2)
    # print(f"{city} 獲勝者是：{winner}，與第二名的得票差距是 {votes_difference}%") # 檢查用
    all_difference[city]  =(winner,votes_difference)
    
max_difference_city = max(all_difference, key = all_difference.get)
max_difference = all_difference[max_difference_city][1]

# print(f"最大得票數差距是 {max_difference}％") # 檢查用

colors ={}

for (city, votes_difference) in all_difference.items():
    ratio = votes_difference[1] / max_difference
    if votes_difference[0] == "tpp":
        HSL = (177, 61, int(75 - 40 * ratio))
    elif votes_difference[0] == "dpp":
        HSL = (130, 60, int(75 - 40 * ratio))
    elif votes_difference[0] == "kmt":
        HSL = (212, 100, int(85 - 40 * ratio))
    # print(f"{city} 獲勝者是：{votes_difference[0]}，與第二名的得票差距是 {votes_difference[1]}%" )
    # print(f"HSL:({HSL[0]},{HSL[1]}%,{HSL[2]}%)")
    color_rgb = ImageColor.getrgb(f"hsl({HSL[0]},{HSL[1]}%,{HSL[2]}%)")
    pprint(color_rgb)
    color_HEX = format(color_rgb[0],"x").zfill(2)+format(color_rgb[1],"x").zfill(2)+format(color_rgb[2],"x").zfill(2)
    pprint(color_HEX)
    colors[city] = f"#{color_HEX}"
    
pprint(colors)

# 開啟 SVG 檔案
file_path = "Blank_Taiwan_map.svg"  # 上傳的檔案路徑
with open(file_path, "r", encoding="UTF-8") as file:
    svg_content = file.read()

# 解析SVG內容
svg_tree = ET.fromstring(svg_content)

# 尋找所有 <path> 元素，檢查其 <title> 子元素是否匹配指定的縣市名稱
for path in svg_tree.iterfind(".//{http://www.w3.org/2000/svg}path"):
    title = path.find("{http://www.w3.org/2000/svg}title")

    if title is not None:
        target = title.text.strip().split(" ")[0]
        path.set("fill", colors[target])

# 如果成功修改，則保存修改後的SVG內容到一個新檔案
modified_svg_path = "Modified_Taiwan_map.svg"
ET.ElementTree(svg_tree).write(modified_svg_path, encoding="utf-8")

# 顯示成功訊息
print(f"修改成功! SVG 檔已存於 {modified_svg_path}")

        

(120, 221, 137)
'78dd89'
(149, 228, 162)
'95e4a2'
(137, 225, 151)
'89e197'
(145, 227, 159)
'91e39f'
(129, 223, 144)
'81df90'
(47, 188, 70)
'2fbc46'
(63, 207, 87)
'3fcf57'
(168, 209, 255)
'a8d1ff'
(112, 179, 255)
'70b3ff'
(129, 223, 144)
'81df90'
(163, 206, 255)
'a3ceff'
(84, 212, 105)
'54d469'
(67, 208, 91)
'43d05b'
(75, 210, 98)
'4bd262'
(80, 211, 101)
'50d365'
(31, 135, 255)
'1f87ff'
(51, 146, 255)
'3392ff'
(141, 226, 155)
'8de29b'
(153, 201, 255)
'99c9ff'
(149, 228, 162)
'95e4a2'
(96, 215, 116)
'60d774'
(0, 114, 245)
'0072f5'
(0, 107, 230)
'006be6'
{'南投縣': '#a3ceff',
 '嘉義市': '#60d774',
 '嘉義縣': '#43d05b',
 '基隆市': '#99c9ff',
 '宜蘭縣': '#50d365',
 '屏東縣': '#4bd262',
 '彰化縣': '#81df90',
 '新北市': '#89e197',
 '新竹市': '#95e4a2',
 '新竹縣': '#a8d1ff',
 '桃園市': '#91e39f',
 '澎湖縣': '#8de29b',
 '總計': '#78dd89',
 '臺中市': '#81df90',
 '臺北市': '#95e4a2',
 '臺南市': '#2fbc46',
 '臺東縣': '#3392ff',
 '花蓮縣': '#1f87ff',
 '苗栗縣': '#70b3ff',
 '連江縣': '#006be6',
 '金門縣': '#0072f5',
 '雲林縣': '#54d469',
 '高雄市': '#3fcf57'}
修改成功! 